# Toric Code Sandbox

In [2]:
from pyquil import Program
from pyquil.gates import *
from pyquil.quil import address_qubits
from pyquil.quilatom import QubitPlaceholder
from pyquil.api import QVMConnection
from pyquil.api import WavefunctionSimulator

import networkx as nx 
from toric_code import * 

from matplotlib import rc 
rc('text', usetex = True)
%matplotlib inline

## Testing Syndrome Extraction 

Following the syndrom extraction protocol presented in Wang et al. (2009) 

In [ ]:
def test_Z_syndrome_extraction(): 
    """ Tests the Z syndrome extraction protocol. 
    """
    pq = Program() 
    ro = pq.declare('ro', 'BIT', 1)
    
    # Initialize the ancilla
    ancilla = QubitPlaceholder() # ==> ancilla = |0> 
    
    # Initialize the data qubits 
    # We'll use the state |0000> + |1111>, which 
    # is a +1 eigenstate of the X stabilizer generator 
    north = QubitPlaceholder()
    south = QubitPlaceholder() 
    east = QubitPlaceholder() 
    west = QubitPlaceholder()
    
    pq += H(north)
    pq += CNOT(north, south)
    pq += CNOT(north, east) 
    pq += CNOT(north, west)
    
    register = [ancilla, north, south, east, west]
    
    # Errors, if appropriate 
    #pq += X(north) # ==> Error detected 
    #pq += X(east) # ==> Errors not detected; state still in codespace 
        
    # Perform the circuit 
    pq += CNOT(north, ancilla)
    pq += CNOT(west, ancilla)
    pq += CNOT(east, ancilla)
    pq += CNOT(south, ancilla)
    
    # Measure in the Z basis 
    pq += MEASURE(ancilla, ro[0])
    return pq 

In [ ]:
qvm = QVMConnection()
pq = test_Z_syndrome_extraction()
pq = address_qubits(pq)
wf_sim = WavefunctionSimulator()
wavefunction = wf_sim.wavefunction(pq)
print(wavefunction)

results = qvm.run(pq, trials=10)
print(results)

In [5]:
def test_X_syndrome_extraction(): 
    """ Tests the X syndrome extraction protocol. 
    """
    pq = Program() 
    ro = pq.declare('ro', 'BIT', 1)
    
    # Initialize the ancilla
    ancilla = QubitPlaceholder()
    pq += H(ancilla) # ==> ancilla = |+> 
    
    # Initialize the data qubits 
    # We'll use the state |0000> + |1111>, which 
    # is a +1 eigenstate of the X stabilizer generator 
    north = QubitPlaceholder()
    south = QubitPlaceholder() 
    east = QubitPlaceholder() 
    west = QubitPlaceholder()
    
    pq += H(north)
    pq += CNOT(north, south)
    pq += CNOT(north, east) 
    pq += CNOT(north, west)
    
    register = [north, south, east, west]
    
    # Errors, if appropriate 
    pq += Z(north) # ==> Error detected 
    #pq += Z(east) # ==> Errors not detected; state still in codespace 
        
    # Perform the circuit 
    pq += CNOT(ancilla, north)
    pq += CNOT(ancilla, west)
    pq += CNOT(ancilla, east)
    pq += CNOT(ancilla, south)
    
    # Measure in the X basis 
    pq += H(ancilla)
    pq += MEASURE(ancilla, ro[0])
    return pq 

In [7]:
qvm = QVMConnection()
pq = test_X_syndrome_extraction()

pq = address_qubits(pq)
wf_sim = WavefunctionSimulator()
wavefunction = wf_sim.wavefunction(pq)
print(wavefunction)

results = qvm.run(pq, trials=1)
print(results[0][0])

(0.7071067812+0j)|00001> + (-0.7071067812+0j)|11111>
1


### Test the syndrome extraction functions for the code 

In [15]:
initialize = Program() 
north = QubitPlaceholder()
south = QubitPlaceholder() 
east = QubitPlaceholder() 
west = QubitPlaceholder()
# Construct a +1 eigenstate of the stabilizer generators 
initialize += H(north)
initialize += CNOT(north, south)
initialize += CNOT(north, east) 
initialize += CNOT(north, west)
# Add a single phase flip error 
initialize += Z(east) # ==> Catches this error! 
# Initialize ancilla to |0> state 
ancilla = QubitPlaceholder()
# Complete the syndrome extraction program
qubits = [ancilla, north, west, east, south]
syndrome = initialize + X_syndrome_extraction(qubits)

# Run the program to obtain the syndrome
qvm = QVMConnection()
syndrome = address_qubits(syndrome)
print(syndrome)
result = qvm.run(syndrome, trials=10)
print(result)

H 0
CNOT 0 1
CNOT 0 2
CNOT 0 3
Z 2
DECLARE ro BIT[1]
H 4
CNOT 4 0
CNOT 4 3
CNOT 4 2
CNOT 4 1
H 4
MEASURE 4 ro[0]

[[1], [1], [1], [1], [1], [1], [1], [1], [1], [1]]


I keep getting errors if I try to name unique memory regions in the syndrome_extraction functions. For now, I'll declare a single bit of classical memory labeled ``'ro'``; it will be overwritten each time we run the syndrome_extraction functions. However, I don't anticipate that this will be too much of a problem, as we can simply immediately save the values stored at ``ro[0]`` elsewhere (not ideal, but it'll work for now). 

Proposed syndrome extraction flow: Set up initial state (e.g., eigenstate of stabilizer generators) --> Apply error model (independent bit or phase flip) --> Construct syndrome extraction program, add to initialization + error program --> Identify stabilizer operator if a -1 eigenvalue is returned (that is, if ancilla = |1>)

Some questions: 

- Can we add the syndrome extraction program to the initialization + error program, without affecting other states? (<-- answer to this question should be YES) 

- How do we identify where the error has occured, for the MWPM algorithm? 

### Testing syndrome extraction for two plaquettes on the primal graph 

In [32]:
# Let's entangle the qubits (7 total) acted on by two adjacent X stabilizers, 
# such that the qubits are +1 eigenstates of both operators 
pq_init = Program() 
# Stabilizer 1 qubits
north_1 = QubitPlaceholder()
south_1 = QubitPlaceholder() 
east_1_2 = QubitPlaceholder() # SHARED by both stabilizers! 
west_1 = QubitPlaceholder()
# Stabilizer 2 qubits 
north_2 = QubitPlaceholder()
south_2 = QubitPlaceholder() 
west_2 = QubitPlaceholder()

# Construct initial state as a +1 eigenstate of both stabilizers 
pq_init += H(north_1)
pq_init += CNOT(north_1, south_1)
pq_init += CNOT(north_1, east_1_2)
pq_init += CNOT(north_1, west_1)

pq_init += H(north_2)
pq_init += CNOT(north_2, south_2)
pq_init += CNOT(north_2, east_1_2)
pq_init += CNOT(north_2, west_2)

# ERRORS, if necessary 
# Stabilizer 1 
pq_init += Z(north_1)
# Stabilizer 2 
pq_init += Z(north_2)

# Syndrome extraction for stabilizer 1 
ancilla_1 = QubitPlaceholder() 
qubits_1 = [ancilla_1, north_1, west_1, east_1_2, south_1]
syndrome_1 = pq_init + X_syndrome_extraction(qubits_1)

# Syndrome extraction for stabilizer 2 
ancilla_2 = QubitPlaceholder()
qubits_2 = [ancilla_2, north_2, west_2, east_1_2, south_2]
syndrome_2 = pq_init + X_syndrome_extraction(qubits_2)

qvm = QVMConnection()
# Address qubits, as necessary 
pq_init = address_qubits(pq_init)
syndrome_1 = address_qubits(syndrome_1)
syndrome_2 = address_qubits(syndrome_2) 

wf_sim = WavefunctionSimulator()
wavefunction = wf_sim.wavefunction(pq_init)
print(wavefunction) # NOTE that wavefunction returned is indeed a +1 eigenstate of the operators! 

result_1 = qvm.run(syndrome_1, trials=10)
result_2 = qvm.run(syndrome_2, trials=10)
print(result_1)
print(result_2)

(0.5+0j)|0000000> + (-0.5+0j)|0001111> + (-0.5+0j)|1110100> + (0.5+0j)|1111011>
[[1], [1], [1], [1], [1], [1], [1], [1], [1], [1]]
[[1], [1], [1], [1], [1], [1], [1], [1], [1], [1]]
